In [84]:
import sys
import copy
sys.path.append('../')
import os
import pandas as pd
from data import transform_id_to_team


In [119]:
season = "2020-21"
print("create training data version 1.0 stable")
# Import data from previous seasons
wdPATH = os.getcwd()

with open(os.path.join(wdPATH, "..", "data_files", "2020-21", "gws", "merged_gw.csv")) as file:
    player_gw = pd.read_csv(file)

player_gw = player_gw.rename(columns={'value': 'now_cost', 'total_points': 'event_points'})
# Player form + ability:
variables_to_keep = ['name', 'team', 'ict_index', 'bps', 'minutes', 'now_cost', 'event_points', 'round'] # should be able to change this in a config file
players_df = player_gw[variables_to_keep]

players_df = players_df.rename(columns={'team': 'team_name'})

# Fixture difficulty:
# Get upcoming gameweek:
with open(os.path.join(wdPATH, "..", "data_files", season, "fixtures.csv")) as file:
    fixture_df = pd.read_csv(file)
    
season_team_df = transform_id_to_team(season) # df containing a name to each team associated with the id for the specified season
players_df = pd.merge(players_df, season_team_df, how="left", on="team_name")
# TO DO: Create a for loop attaching the FDR for each player for each round to the df    

# 2. Separate home and away team FDR and append them into two columns
fixture_a = fixture_df[["team_a", "team_a_difficulty", "event"]]
fixture_a = fixture_a.rename(columns={'team_a': 'team', 'team_a_difficulty': 'fdr', 'event': 'round'})
fixture_h = fixture_df[["team_h", "team_h_difficulty", "event"]]
fixture_h = fixture_h.rename(columns={'team_h': 'team', 'team_h_difficulty': 'fdr', 'event': 'round'})
fixture_difficulty = fixture_a.append(fixture_h)

fixture_difficulty = fixture_difficulty.rename(columns={'team': 'team_id'})

# 3. Merge the FDR to the main dataframe
main_df = pd.merge(players_df, fixture_difficulty, how="left", on=['team_id', "round"])

# 4. Remove team and team_id as a feature to predict on to create a more generalized dataset to train on. 
variables_to_keep = ['name','ict_index', 'bps', 'minutes', 'now_cost', 'team_id', 'fdr', 'round', 'event_points'] # should be able to change this in a config file
main_df = main_df[variables_to_keep]

create training data version 1.0 stable


In [125]:
main_df

,name,ict_index,bps,minutes,now_cost,team_id,fdr,round,event_points,avg_minutes,ict_index_change,bps_change
0,Aaron Connolly,3.4,-3,45,55,3,4,1,1,0,0,0
1,Aaron Cresswell,2.2,11,90,50,19,2,1,1,0,0,0
2,Aaron Mooy,0.0,0,0,50,3,4,1,0,0,0,0
3,Aaron Ramsdale,1.8,12,90,50,15,2,1,1,0,0,0
4,Abdoulaye DoucourÃ©,6.9,20,90,55,7,4,1,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27390,Aaron Ramsdale,3.2,30,90,46,15,2,38,9,0,0,0
27391,Adrien Silva,0.0,0,0,44,9,3,38,0,0,0,0
27392,Sidnei Tavares,0.0,0,0,44,9,3,38,0,0,0,0
27393,Kalvin Phillips,6.1,17,90,49,10,2,38,6,0,0,0


In [153]:
main_df.loc[main_df['round'] == 35]

,name,ict_index,bps,minutes,now_cost,team_id,fdr,round,event_points,avg_minutes,ict_index_change,bps_change
23569,Max Thompson,0.0,0,0,45,4,2,35,0,0,0,0
23570,Willy Boly,0.0,0,0,54,20,3,35,0,0,0,0
23571,Ciaran Clark,0.0,0,0,45,14,4,35,0,0,0,0
23572,Luke Cundle,0.0,0,0,45,20,3,35,0,0,0,0
23573,Matt Ritchie,7.8,32,90,49,14,4,35,8,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
25398,Aaron Ramsdale,5.1,22,90,46,15,2,35,3,0,0,0
25399,Adrien Silva,0.0,0,0,44,9,4,35,0,0,0,0
25403,Sidnei Tavares,0.0,0,0,44,9,4,35,0,0,0,0
25407,Kalvin Phillips,0.1,2,1,49,10,3,35,1,0,0,0


In [149]:
main_df.drop_duplicates(subset=["name", "round"], keep="first", inplace=True)

In [152]:
main_df

,name,ict_index,bps,minutes,now_cost,team_id,fdr,round,event_points,avg_minutes,ict_index_change,bps_change
0,Aaron Connolly,3.4,-3,45,55,3,4,1,1,0,0,0
1,Aaron Cresswell,2.2,11,90,50,19,2,1,1,0,0,0
2,Aaron Mooy,0.0,0,0,50,3,4,1,0,0,0,0
3,Aaron Ramsdale,1.8,12,90,50,15,2,1,1,0,0,0
4,Abdoulaye DoucourÃ©,6.9,20,90,55,7,4,1,3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27390,Aaron Ramsdale,3.2,30,90,46,15,2,38,9,0,0,0
27391,Adrien Silva,0.0,0,0,44,9,3,38,0,0,0,0
27392,Sidnei Tavares,0.0,0,0,44,9,3,38,0,0,0,0
27393,Kalvin Phillips,6.1,17,90,49,10,2,38,6,0,0,0


In [129]:
len(test_df)

44

In [154]:
main_df['avg_minutes'] = 0
main_df['ict_index_change'] = 0
main_df['bps_change'] = 0

In [155]:
output_df = pd.DataFrame()

In [156]:
enter_df = copy.copy(main_df)
enter_df = enter_df.sort_values(by=['name', 'round'])

In [157]:
enter_df

,name,ict_index,bps,minutes,now_cost,team_id,fdr,round,event_points,avg_minutes,ict_index_change,bps_change
0,Aaron Connolly,3.4,-3,45,55,3,4,1,1,0,0,0
425,Aaron Connolly,6.9,27,89,55,3,2,2,8,0,0,0
964,Aaron Connolly,1.9,2,73,55,3,4,3,2,0,0,0
1518,Aaron Connolly,0.7,7,65,55,3,3,4,2,0,0,0
2080,Aaron Connolly,3.0,13,12,55,3,3,5,4,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
22578,Ã‡aglar SÃ¶yÃ¼ncÃ¼,2.3,13,90,53,9,2,33,2,0,0,0
23146,Ã‡aglar SÃ¶yÃ¼ncÃ¼,4.7,15,90,53,9,2,34,2,0,0,0
24219,Ã‡aglar SÃ¶yÃ¼ncÃ¼,3.2,10,90,53,9,4,35,0,0,0,0
26215,Ã‡aglar SÃ¶yÃ¼ncÃ¼,2.3,14,90,53,9,4,37,1,0,0,0


In [ ]:
# Takes forever to run (approx 1hr 15 min)
for name in enter_df['name']:
    input_df = copy.copy(enter_df)
    input_df = input_df.loc[input_df['name'] == name]
    for gw in input_df['round']:
        input_df['avg_minutes'] = input_df["minutes"].mean()
        if gw > 1:
            input_df['ict_index_change'] = input_df["ict_index"].diff(periods=1)
            input_df['bps_change'] = input_df["bps"].diff(periods=1)
    output_df = output_df.append(input_df)
    

In [ ]:
for name in enter_df['name']:
    input_df = copy.copy(enter_df)
    input_df = input_df.loc[input_df['name'] == name]
    input_df['avg_minutes'] = input_df['minutes'].expanding().mean()
    output_df = output_df.append(input_df)
    

In [158]:
output_df = enter_df.groupby(['name'])['minutes'].expanding().mean().reset_index(0)

In [159]:
output_df['index_id'] = output_df.index

In [160]:
enter_df['index_id'] = enter_df.index

In [161]:
merged_df = enter_df.merge(output_df, how="left", on=["index_id"])

In [173]:
merged_df

,name,ict_index,bps,minutes,now_cost,team_id,fdr,round,event_points,avg_minutes,ict_index_change,bps_change,index_id,avg_minutes
0,Aaron Connolly,3.4,-3,45,55,3,4,1,1,0,0,0,0,45.000000
1,Aaron Connolly,6.9,27,89,55,3,2,2,8,0,0,0,425,67.000000
2,Aaron Connolly,1.9,2,73,55,3,4,3,2,0,0,0,964,69.000000
3,Aaron Connolly,0.7,7,65,55,3,3,4,2,0,0,0,1518,68.000000
4,Aaron Connolly,3.0,13,12,55,3,3,5,4,0,0,0,2080,56.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22870,Ã‡aglar SÃ¶yÃ¼ncÃ¼,2.3,13,90,53,9,2,33,2,0,0,0,22578,41.161290
22871,Ã‡aglar SÃ¶yÃ¼ncÃ¼,4.7,15,90,53,9,2,34,2,0,0,0,23146,42.687500
22872,Ã‡aglar SÃ¶yÃ¼ncÃ¼,3.2,10,90,53,9,4,35,0,0,0,0,24219,44.121212
22873,Ã‡aglar SÃ¶yÃ¼ncÃ¼,2.3,14,90,53,9,4,37,1,0,0,0,26215,45.470588


In [170]:
merged_df = merged_df.drop("name_y", 1)


In [172]:
merged_df = merged_df.rename(columns={'name_x': 'name', 'minutes_x': 'minutes', 'minutes_y': 'avg_minutes'})

In [178]:
merged_df

,name,ict_index,bps,minutes,now_cost,team_id,fdr,round,event_points,avg_minutes,ict_index_change,bps_change,index_id,avg_minutes
0,Aaron Connolly,3.4,-3,45,55,3,4,1,1,0,0,0,0,45.000000
1,Aaron Connolly,6.9,27,89,55,3,2,2,8,0,0,0,425,67.000000
2,Aaron Connolly,1.9,2,73,55,3,4,3,2,0,0,0,964,69.000000
3,Aaron Connolly,0.7,7,65,55,3,3,4,2,0,0,0,1518,68.000000
4,Aaron Connolly,3.0,13,12,55,3,3,5,4,0,0,0,2080,56.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22870,Ã‡aglar SÃ¶yÃ¼ncÃ¼,2.3,13,90,53,9,2,33,2,0,0,0,22578,41.161290
22871,Ã‡aglar SÃ¶yÃ¼ncÃ¼,4.7,15,90,53,9,2,34,2,0,0,0,23146,42.687500
22872,Ã‡aglar SÃ¶yÃ¼ncÃ¼,3.2,10,90,53,9,4,35,0,0,0,0,24219,44.121212
22873,Ã‡aglar SÃ¶yÃ¼ncÃ¼,2.3,14,90,53,9,4,37,1,0,0,0,26215,45.470588


In [202]:
input_df = pd.DataFrame()
output_df = pd.DataFrame()
namelist = list()

for name in merged_df['name']:
    if name not in namelist:
        namelist.append(name)
        input_df = merged_df
        input_df = input_df.loc[input_df['name'] == name]
        input_df['event_points'] = input_df['event_points'].shift(1)
        output_df = output_df.append(input_df)

C:\Users\magnu\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [203]:
output_df['ict_index_change'] = output_df["ict_index"].diff(periods=1)
output_df['bps_change'] = output_df["bps"].diff(periods=1)

In [200]:
output_df = output_df.loc[output_df['name'] == 'Bernd Leno']

In [206]:
output_df

,name,ict_index,bps,minutes,now_cost,team_id,fdr,round,event_points,avg_minutes,ict_index_change,bps_change,index_id,avg_minutes
1,Aaron Connolly,6.9,27,89,55,3,2,2,1.0,0,3.5,30.0,425,67.000000
2,Aaron Connolly,1.9,2,73,55,3,4,3,8.0,0,-5.0,-25.0,964,69.000000
3,Aaron Connolly,0.7,7,65,55,3,3,4,2.0,0,-1.2,5.0,1518,68.000000
4,Aaron Connolly,3.0,13,12,55,3,3,5,2.0,0,2.3,6.0,2080,56.800000
5,Aaron Connolly,0.0,0,0,55,3,2,6,4.0,0,-3.0,-13.0,2667,47.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22870,Ã‡aglar SÃ¶yÃ¼ncÃ¼,2.3,13,90,53,9,2,33,6.0,0,0.3,-12.0,22578,41.161290
22871,Ã‡aglar SÃ¶yÃ¼ncÃ¼,4.7,15,90,53,9,2,34,2.0,0,2.4,2.0,23146,42.687500
22872,Ã‡aglar SÃ¶yÃ¼ncÃ¼,3.2,10,90,53,9,4,35,2.0,0,-1.5,-5.0,24219,44.121212
22873,Ã‡aglar SÃ¶yÃ¼ncÃ¼,2.3,14,90,53,9,4,37,0.0,0,-0.9,4.0,26215,45.470588


In [205]:
output_df = output_df.dropna()

In [ ]:
if round > 1:
                input_df['ict_index_change'] = input_df["ict_index"].diff(periods=1)
                input_df['bps_change'] = input_df["bps"].diff(periods=1)

In [65]:
import numpy as np

In [69]:
enter_df['event_points'] = np.where(
    enter_df['name'] == enter_df['name'].shift(1),
    enter_df['event_points'].shift(1),
    #else:
    0
)

In [61]:
enter_df['event_points'] = enter_df.groupby(['name'])['event_points'].shift(1)

In [70]:
enter_df

,name,ict_index,bps,minutes_x,now_cost,team_id,fdr,round,event_points,avg_minutes,ict_index_change,bps_change,minutes_y
0,Aaron Connolly,3.4,-3,45,55,3,4,1,0.0,0,0,0,45.000000
1,Aaron Connolly,3.4,-3,45,55,3,4,1,NaN,0,0,0,67.000000
2,Aaron Connolly,3.4,-3,45,55,3,4,1,NaN,0,0,0,69.000000
3,Aaron Connolly,3.4,-3,45,55,3,4,1,NaN,0,0,0,68.000000
4,Aaron Connolly,3.4,-3,45,55,3,4,1,1.0,0,0,0,56.800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120836,Ã‡aglar SÃ¶yÃ¼ncÃ¼,4.3,16,90,53,9,3,38,0.0,0,0,0,46.325000
1120837,Ã‡aglar SÃ¶yÃ¼ncÃ¼,4.3,16,90,53,9,3,38,0.0,0,0,0,47.390244
1120838,Ã‡aglar SÃ¶yÃ¼ncÃ¼,4.3,16,90,53,9,3,38,0.0,0,0,0,48.404762
1120839,Ã‡aglar SÃ¶yÃ¼ncÃ¼,4.3,16,90,53,9,3,38,0.0,0,0,0,49.372093
